<div style="float:left">
    <h1 style="width:600px">Workshop 4: API and Data Munging</h1>
    <h3 style="width:600px">CASA0006: Data Science for Spatial Systems</h3>
    <h3 style="width:600px">CASA0009: Spatial Data Capture, Storage and Analysis</h3>
    <h3 style="width:600px">Author: Huanfa Chen</h3>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

## Introduction

In this workshop, you will practice the following skills:

1. Accessing the weather data via APIs 
2. Transforming the weather data into a Pandas DataFrame and exploring the dataset using the Pandas library.

## Preparation

### Installing the computing environment

Before the workshop, please install the SDS (short for **Spatial Data Science**) computing environment on your computer, following this [link](https://github.com/jreades/sds_env/). This environment would be used in the workshops and the final coursework of CASA0006.

Note that this environment is the same as the one used in CASA0013 and CASA0007. You don't need to install it again if you have done so in one of these modules in Term 1.

### Getting to grips with Jupyter and Markdown

If you are not familiar with Jupyter and Markdown (and Git), please go through this practical which is part of the CASA0013 module: https://github.com/jreades/i2p/blob/master/practicals/Practical-01-Getting_Started.ipynb

### Learning Python

If you are not familiar with Python, here is a few good resources that teach Python:

- Code Camp: https://bit.ly/33Rc5QD
- Short Python courses on Linkedin learning: https://www.linkedin.com/learning/search?keywords=python&u=69919578

## The importance of exploration

One of the first things that we do when working with any new data set is to familiarise ourselves with it. There are a _huge_ number of ways to do this, but there are no shortcuts to:
* Reading about the data (how it was collected, what the sample size was, etc.)
* Reviewing any accompanying metadata (data about the data, column specs, etc.)
* Looking at the data itself at the row- and column-levels
* Producing descriptive statistics 
* Visualising the data using plots 
In fact, you should use _all_ of these together to really understand where the data came from, how it was handled, and whether there are gaps or other problems. If you're wondering which comes first, I've always liked this approach: _start with a chart_. We're _not_ going to do that here because, first, I want you to get a handle on pandas itself!

## Pandas with Weather API Data 

The UK's Met Office is a world-leading weather and climate research centre, and even if it doesn't always seem like their forecasts are very accurate that's because Britain's weather is inherently _unpredictable_. They've also done a lot of work to make their weather data widely available to people like us.

I probably don't need to say a _lot_ about weather data because you've probably been making use of forecasts for much of your life! But it's _still_ worth understanding something about how weather data is gathered and reported: many organisations operate weather stations where data on wind speed, temperature, rain, and amount of sun are collected and then transmitted to a server to be integrated into a larger data set of weather _observations_ at a national or global scale. Of course, any _one_ station might be in the 'wrong' place (somewhere shady or protected from the rain) or it might even break down, but the idea is that if you have enough of them you can collect a pretty good range of data for the country and begin to look for patterns and, potentially, make predictions.

We will be accessing data from the MetOffice from weather stations where observations, such as the ones below, are collected:
* <Param name="F" units="C">Feels Like Temperature (units: degrees Celsius)
* <Param name="G" units="mph">Wind Gust (units: mph)</Param>
* <Param name="H" units="%">Screen Relative Humidity (units: percent)</Param> 
* <Param name="T" units="C">Temperature (units: degrees Celsius)</Param> 
* <Param name="V" units="">Visibility (units: km?)</Param> 
* <Param name="D" units="compass">Wind Direction (units: compass degrees)</Param>  
* <Param name="S" units="mph">Wind Speed (units: mph)</Param> 
* <Param name="U" units="">Max UV Index (units: index value)</Param> 
* <Param name="W" units="">Weather Type (units: categorical)</Param> 
* <Param name="Pp" units="%">Precipitation Probability (units: percent)</Param>

These observations are associated with a particular station (where did we see these values/where _will_ we see these values?), they will also be associated with _either_ a particular time in the past (when were they collected?) or, if they're forecasts, with a particular time in the future (when do we expect to see them?). 

So although weather data might seem more 'objective' than data on social class (though for obvious reasons it turns out that both are just attempts to capture data about reality, not reality itself), it may also turn out to be very complex to store and manage beccause of the temporal element _and_ the fact that it's not just a count of one thing, each of these observations uses a very different set of units.

To really get to grips with the MetOffice API you will need to RTM (Read The Manual): http://www.metoffice.gov.uk/media/pdf/3/0/DataPoint_API_reference.pdf. The ruder version of that, which you will sometimes see on StackOverflow and elsewhere, is: RTFM.

## Getting Weather Data via an API

Because the weather is changing all the time, so is the data! And, 'worse', it's becoming obsolete: the forecast from 2 years ago isn't particularly useful to us now. *And* asking for "yesterday's weather" depends on the day that we're asking! When you have data that is always changing from minute to minute or day to day then you use an API (Application Programming Interface) to access it: the API knows that "yesterday's weather" means "work out what day it is right now and then get the weather from the day before", and it also knows that "give me the current weather from station X" means "look up station X and find the latest weather report that I've received". In other words, an API is  designed with programmatic, dynamic interaction in mind right from the start.

### So What _is_ an API?

There's a nice, friendly introduction to APIs over at [Free Code Camp](https://medium.freecodecamp.com/what-is-an-api-in-english-please-b880a3214a82#.rmjnli2nn). You can combine this with watching a video from MuleSoft that compares an API with a waiter in a restaurant:

[![What is an API?](http://img.youtube.com/vi/s7wmiS2mSXY/0.jpg)](https://www.youtube.com/watch?v=s7wmiS2mSXY)

Helpfully, the MetOffice provides a lot of documentation about their API (I'd suggest bookmarking it): http://www.metoffice.gov.uk/datapoint/support/api-reference

This type of data requires a lot more research up front to work with, but it's very flexible once you know how to 'speak API' because you can _customise_ the API request so that the server responds with _only_ the data we're interested in instead of being 'stuck' with what the provider wants to give us.

### Obtaining an API Key

The first step to working with the API from the MetOffice is to obtain an API key: [do that here](http://www.metoffice.gov.uk/datapoint/API). 

Make a note of this key in your notebook. Right here:

In [ ]:
api_key   = "" # your API key

That way your API key is saved somewhere easy to access.

We _always_ have to use the key as part of an API request: the process by which we _ask_ for data. Think of the key as being _your_ unique identifier: no two people share the same key and that way the MetOffice can cut off people who abuse the system or look at which APIs are popular with lots of users... Twitter and Facebook do the same thing.

## Obtaining a List of Sites from the API

How to start? Well, the first thing that we need to know is: for what locations can I get weather data? For this to work, we need to know how to ask the API nicely for a list of available sites... We're going to show you the code and output _first_ and then we'll step through what's involved. You won't see any output from the code in the next block because we look at that later.

In [ ]:
import json, requests # Libraries we need

api_url  = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
site_url = "val/wxfcs/all/json/sitelist" # sites API URL

payload = {'key': api_key} # Dictionary to hold request parameters
s = requests.get(api_url + site_url, params=payload) # Do the request

sites = s.json() # Capture the output

print("Done!")

OK, now let's try making sense of the JSON code:
```python
import json, requests
```
So, first we import two new modules: one that makes requests to a web server, and one that will parse JSON\* responses from the server in order to turn them into something that we can work with more easily.

\* We'll get to what JSON is in a second.

```python
api_url  = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
site_url = "val/wxfcs/all/json/sitelist" # sites API URL
```

Then we set up some default variables (`api_url`, `site_url`) that will help us to build our request to the MetOffice's server. The comments help us to remember what each of these variables holds.

```python
payload = {'key': api_key} # Dictionary to hold request parameters
s = requests.get(api_url + site_url, params=payload) # Do the request
```

You'll notice that the `payload` is just a dictionary and that this dictionary is then passed to the `requests` library (the `get` function). All it does it convert this dictionary to a key-value pair in the format _expected by the API_. Think of it as a kind of translation between languages: from the language of Python to the language of the web (HTTP, to be precise).

We issue our request and it returns a response that we store in `s` (short for `sites data`).

```python
sites = s.json() # Capture the output
```

Lastly, we ask the response object to convert the reply into a JSON data structure... more on JSON in a second, but first let's look at what we got from our request!

In [ ]:
# Show the requested URL
print(s.url) # Click on the link below to see it nicely formatted automatically!

In [ ]:
# what is the type of *sites*
print(type(sites))
# print it. It would be lengthy and confusing
print(sites)

### So, What's JSON?

It's pretty hard to figure out what that 'JSON' reply means, but if you look closely you'll start to see a few things that you recognise:

1. There is the '{' and '}' that we associate with dictionaries
2. There is the ':' that we associate with key-value pairs in dictionaries
3. The really hard thing is all the extra 'u's all over the place but that's something that's confusing to _us_, not the _computer_.

So it's actually just a kind of dictionary containing lists and other dictionaries. That's it. The output looks like a mess, but it _is_ a dictionary and the only thing that is entirely new is the fact that every string has the letter 'u' in front of it. 

That 'u' means 'Unicode' and it just a special kind of string that supports accents, Chinese characters, emojis, and just about anything else that you can think of...

### Using XML to better understand JSON

We want to work with JSON replies because they are easier for Python to read, but sometimes it is easier for _us_ to read the output of an API if we are able to ask for the answer in XML (eXtensible Markup Language). Try running the code block below and then copy+pasting the URL that it outputs into your browser window:

In [ ]:
xml_url = "val/wxfcs/all/xml/sitelist"
x = requests.Request('GET', api_url + xml_url, params=payload).prepare()
print(x.url)

### Back to JSON...

Now that you have a better idea of what data the reply contains, let's see if we can convert the JSON reply into something useful for Python; if you scroll back up to where we printed out the reply you'll notice that it all starts with a '{', meaning that it's a dictionary.

Let's start by printing out the keys in the dictionary and the _type_ of data associated as a value to that key:

In [ ]:
for k in sites.keys():
    print("Key: " + str(k))
    print("Value: " + str(type(sites[k])))

Not the most exciting answer, but at least we know that the value is a dictionary. Let's try moving down a level:

In [ ]:
for k in sites.keys():
    print("Key: " + str(k))
    print("Value: " + str(type(sites[k])))
    for k2 in sites[k].keys():
        print("\tKey: " + str(k2))
        print("\tValue: " + str(type(sites[k][k2])))

The MetOffice is not making life easy for us here: there's a _lot_ of extra 'baggage' in this API response. But we at least know that the next level down is a list and _that_ suggests that things are about to get a bit more interesting... Let's simplify our code at the same time:

In [ ]:
apiList = sites['Locations']['Location']
print("List in API response is " + str(len(apiList)) + " long")

Now _that_ is a rather more interesting response. What it means is that our JSON reply has this structure:
```
{
    'Locations': {
        'Location': [
            ... lots of data here ...
        ]
    }
}
```

If you scroll back up to the JSON reply you should now be able to read a little bit more of the response... and this should give you a clue as to how to print out the `name`, `id`, and `longitude` of the first five sites. I'll get you started:

In [ ]:
for i in range(???):
    location = apiList[???]
    print("Location: " + ??? + " (id: " + ??? + ") is at longitude: " + location[???])

Hint: in the Line 3 of the previous cell, please replace the ```???``` with one of the following terms:
- ```location['name']```
- ```location['longitude']```
- ```location['id']```

Your answer should look like this:

```python
Location: Carlisle Airport (id: 14) is at longitude: -2.8092
Location: Liverpool John Lennon Airport (id: 26) is at longitude: -2.85
Location: Scatsta (id: 33) is at longitude: -1.2992
Location: Kinloss (id: 3066) is at longitude: -3.5606
Location: Lossiemouth (id: 3068) is at longitude: -3.322
````

And _now_ that we know where all the data was 'hidden', we can convert this to a proper data structure in which it is possible to _interact_ with it. To do that, we'll put the site data into a pandas data frame...

# Turning API data into a Pandas DataFrame

Pandas is remarkably intelligent and will _often_ -- though not always -- work out the sensible thing to do from many kinds of data structures (list-of-lists, dictionary-of-lists, list-of-dictionaries...). So let's see what happens when we simply pass `apiList` (a LoD) directly to the `DataFrame` 'constructor' instead of passing the data through, for instance, the `read_csv` function as we did above with a CSV file.

In [ ]:
import pandas as pd
df2 = pd.DataFrame(apiList)

df2.head()

Wow, that's... almost scarily easy. You can see that pandas worked out the structure of our LoD and then automatically converted that to columns in a data frame. So it got the hardest part of the process exactly right and has saved us a lot of work. _That_ is the point of functions and of code: to be constructively lazy.

Of course, we could have predicted that pandas would cope since there is a whole section in the documentaiton [devoted to creating data frames from different structures](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe).

But let's take a slightly closer look at the Elevation data:

In [ ]:
df2.elevation.describe()

What's that? The data-type (dtype) is an _object_? Shouldn't that be a float? And shouldn't the description give us the 7-figure summary for numeric data? The 'problem' (if you looked closely at the JSON response) is that everything came back to us as Unicode text (the little 'u's in front of each key and value) and not as numbers.

## Fixing Column Data-types

The problem is that pandas didn't know what we expected the columns to be, so it's treated them all as 'objects' (basically: strings) and not as numeric data types. To fix that you need to know that there's a function called `'astype'` that allows pandas to convert between different data types where it's fairly easy for pandas to figure out what we want to do:

In [ ]:
for c in ['region','unitaryAuthArea']:
    df2[c] = df2[c].astype('category')
for c in ['elevation','latitude','longitude']:
    df2[c] = df2[c].astype('float')
for c in ['id']:
    df2[c] = df2[c].astype('int')

Let's try that description again:

In [ ]:
df2.elevation.describe()

In [ ]:
%matplotlib inline
df2.elevation.plot.hist()

### Investigating Unknown Columns

You'll note that I didn't change the column types for `name`, `nationalPark`, and `obsSource`. There's no reason to change the name since we rather _assume_ that it's pretty much unique (the output of `describe` confirms it). For the others, it's because we don't really know what we expect to find!

There's no 'one' way to investigate the values in an 'unknown' column, but it's often worth thinking about _unique_ values as a starting point.

In [ ]:
print("Data type is: " + df2.nationalPark.dtype.name)
print(" ")
print(df2.nationalPark.describe())

This tells us that there are 618 rows (out of nearly 6,000) that have a value that is _not_ `NaN`, and that there are 16 unique non-`NaN` values. We can list them easily in pandas:

In [ ]:
df2.nationalPark.unique()

Given the number of unique parks, what is the _best_ data-type for this column? Fix the code below to change the column type of the nationalPark data and assign the updated column it back to `nationalPark` in the block below:

In [ ]:
df2['???'] = df2.???.astype(???)

And now let's check our data type:

In [ ]:
print("Data type is: " + df2.nationalPark.dtype.name)

# Finding Data in Pandas

We've got ourselves a pandas data frame containing a set of locations, how do we go about finding one or more _specific_ rows in the data set rather than just summarising the data via `describe`?

## Searching for a Number

This is the easiest type of search to do in pandas because it looks _most_ like code you've already seen.

### Find All Sites East of 1.1 Degrees Longitude

To translate this into code, we just need to remember that: a) East would be _greater than_; and b) longitude is already a float. So in that case it's...

In [ ]:
dfEast = df2[df2.longitude > 1.1]
dfEast.head(7)

Let's break this down:

* `df2.longitude` is obviously the longitude column of our data frame `df2`
* `df2.longitude > 1.1` is therefore a kind of _query_ (or _selection_) of rows where the longitude is greater than 1.1. What it _actually_ does is compare each row's longitude value to 1.1 and remember if the result is `True` or `False`.
* `df2[ ... ]` is _like_ what we do with a list when we write: `myList[3:5]` to select the fourth through sixth elements of a list, but in pandas we can _select_ non-sequential rows because we are using a `boolean` array (a.k.a. list) that looks like this: `[False, False, False, True, True, True, ...]`.
* `dfEast = ...` saves the _result_ of the selection into a new data frame called `dfEast` (data frame East).

You can check what I'm saying about the boolean result using:

In [ ]:
df2.longitude > 1.1

And we can check that `dfEast` and `df2` are not the same using `shape`, which gives us the dimensions of the data frame as `(<rows>, <columns>)`:

In [ ]:
print(df2.shape)
print(dfEast.shape)

What this first example means is that _anything_ that can be evaluated to `True` or `False` can be used to select rows from a data frame... Let's try some more selections based on numbers...

### Finding the Minimum & Maximum Elevation

The lowest point is in the Fens, and the highest point is, of course, Ben Nevis:

In [ ]:
df2[df2.elevation == df2.elevation.min()] # Somewhere in Cambridgeshire

In [ ]:
df2[df2.elevation == df2.elevation.???] # Find Ben Nevis

### Finding a Range Between Known Values

Perhaps we aren't just looking for extremes... how about all of the areas between 55 and 55.2 degrees latitude? *[There were 91 the last time I checked.]*

In [ ]:
dfRange = df2.loc[ (df2.latitude > 55.0) & (df2.latitude < 55.2) ]
dfRange

That example contains a few new things to which you need to pay attention:
1. You'll see that, with mutiple selections, we had to put parentheses around each one -- this is to avoid confusing pandas as to what it should do _first_.
2. We see an '&' (ampersand) which is completely new: it's a logical `AND` that asks pandas to "Find all the rows where condition 1 _and_ condition 2 are both `True`". So it calculates the `True`/`False` for the left side and the `True`/`False` for the right side of the `&`, and then combines them. Look at the appendix to this notebook for more examples and options.
3. We had to a `.loc` on the end of the `df2` -- the best way to think of this is that it 'freezes' things so as to prepare the data frame to do a search based on the _location_ of some complex selection criteria. We'll see more of this next week.

### Finding a Range Based on the Distribution

Finally, let's try finding the stations whose elevation is _greater_ than the mean. *[There are 1,330 the last time I checked.]*

In [ ]:
dfMean = df2.loc[ df2.elevation > df2.???.mean() ]
print("There are " + str(dfMean.shape[0]) + " sites above the mean elevation of " + str(df2.elevation.mean()))

## Searching on Text & Categories

Numeric searching is all well and good, but what if I'm interested in finding stations in a particular area?

### Searching for a Category

Let's find the names of every station inside the Cairngorms National Park! *[There were 71 the last time I checked.]*

In [ ]:
df2[ df2.??? == ??? ].name

### Searching for Part of a String

If you want to find a full match for a string then it's fairly easy and works like everything you've seen before with string matching:

In [ ]:
df2[df2.name=='Cairn Gorm Summit']

And pandas also provides a lot of useful tools for searching _inside_ a string, as long as you remember to _tell_ pandas to use the string-methods (notice the format: `<data frame>.<data series>.str.<string method>()`):

In [ ]:
df2[df2.name.str.startswith('Beinn A\'')]

In [ ]:
df2[df2.name.str.endswith('Summit')]

Searching _inside_ a string is no harder:

In [ ]:
df2[df2.name.str.contains('Charn')]

How would you ensure that you found _only_ the Geal Charn stations? 

Combine what you've learned above to create a complex query (two conditions on a single line) using a mix of search critera:

In [ ]:
df2.loc[ (???) & (???'Highland') ]

Finally, I want you to find and print out **_only_ the ID of Heathrow the town _not_ the Airport** using a single line of code. There are _at least_ two ways to retrieve this...

We are going to want that ID for the next step in working with the MetOffice API, but there is a last trick to learn here and that's how to extract an actual value as a string, int, or float from a data series. The thing to remember is that a Series is basically a list with a lot of value-added features. The contents of the list can be found in `<data series>.values`. So to get the 2nd through 5th values of the elevation column it would be:
```python
df2.elevation.values[1:5]
```

If you do the selection criteria for Heathrow Airport properly there should only be one item in the list of values that you retrieve, so the right code will include a `[0]` at the end:

In [ ]:
heathrow = ???
print(heathrow)

# Obtaining Weather Data

The next step in this process is a bit more complicated because weather data is a bit more complicated than a list of locations...

First, just in case you want to only run this section again (and not revisit the content above), I'd suggest saving a copy of your API key here as well:

In [ ]:
api_key = '???'

In [ ]:
import json, requests # Libraries we need

api_url  = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
obs_json= "val/wxobs/all/json/" # observations URL

heathrow = str(3772)  # heathrow town weather station

payload = {'res': 'hourly', 'key': api_key} # Dictionary to hold request parameters

r = requests.get(api_url + obs_json + heathrow, params=payload)

print(r.url)

weather = r.json() # Capture the reply

print("Done!")

In [ ]:
print(weather)

That's yet another bunch of 'data' that's difficult for us to read, but by now this should be looking rather familiar to you... perhaps? Hang on a moment! It's a dictionary-of-lists-of-dictionaries-of...

## Creating a DataFrame from a Dictionary

And that, of course is exactly the type of data structure that we can work with in pandas! 

So the _last_ step here is to figure out how to create a new data frame from this dictionary. Here, the MetOffice has _not_ made our lives very easy because the data is packaged in a way that doesn't allow us to easily load it into pandas. If you search online, you'll find plenty of people complaining about how the MetOffice API works. Or doesn't work, if you prefer.

So we're not going to ask you to sort this out for yourselves. Instead, we're going to provide you with a function (!) to take the observation data and convert it into a data frame.

In [ ]:
from datetime import datetime, timedelta
def processMetOfficeObservations(loc): 
    """
    Process a series of 'reports' for a single
    location using the datetime object as the 
    reference time against which to build the 
    timedelta (i.e. we start from midnight and 
    the timedelta is the number of minutes past 
    midnight)
    """
    observations = {} # Stores results
    
    for d in loc['Period']: # d for day
        dt = datetime.strptime(d['value'],'%Y-%m-%dZ') # Convert date to datetime object
    
        # Now deal with the actual observations (i.e. 'Reports')
        for report in d['Rep']:
            
            # Find the timestampe and add it to the date
            minutes_after_midnight = int(report['$'])
            ts = dt + timedelta(minutes=minutes_after_midnight)
            
            # For each of the possible values, set a default value
            # if the weather station doesn't actually collect that
            # parameter... can you see a problem with our defaults?
            if 'ts' not in observations:
                observations['ts'] = []
            observations['ts'].append( str(ts) )
            for key in ['D','Pt']:
                if key not in report:
                    report[key] = u""
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
            for key in ['W','V','S','G']:
                if key not in report or report[key] == "":
                    report[key] = 0
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
            for key in ['T','Dp','H']:
                if key not in report or report[key] == "":
                    report[key] = 0.0
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
    
    return observations

In [ ]:
import pandas as pd
data = processMetOfficeObservations( weather['SiteRep']['DV']['Location'] )
df3 = pd.DataFrame.from_dict( data )

Well that was hard... but did it really do what we expected?

In [ ]:
df3.head(3) # Check that it did what we expected

## Tidying Up

Before we can get back to plotting (again) we have a few more steps to work through:

1. To rename the columns to something a little more useful.
2. To turn the 'ts' field into an _actual_ timeseries so that pandas understands what it is.
3. To convert all of the other series to the right numerical/categorical format.

Let's do this in several stages... 

### Changing Column Names

You may remember that I indicated what the observations returned by each weather station might include:

* D  = Wind Direction
* Dp = Dew Point
* G  = Wind Gust
* H  = Humidity
* Pt = Pressure Tendency
* S  = Wind Speed
* T  = Temperature
* V  = Visibility
* W  = [Weather Type](https://www.metoffice.gov.uk/services/data/datapoint/code-definitions)
* ts = Time of Day

Given this, and the fact that I've listed these in order, what needs to replace the '???' in the code below?

In [ ]:
column_map = {
    'D':  'WindDirection',
    'Dp': 'DewPoint',
    'G':  'WindGust',
    'H':  'Humidity',
    'Pt': 'PressureTendency',
    'S':  'WindSpeed',
    'T':  'Temperature',
    'V':  'Visibility',
    'W':  'WeatherType',
}
df3.rename(columns=???, inplace=True)
df3.head(3)

You should see the 'full' column names now.

### Changing column types

If you were exploring the data frame along the way, you might have already noticed that the description of numeric columns (like Temperature) doesn't seem much like what we had before -- shouldn't we get the 7-figure summary for numeric columns? The problem is that pandas didn't know what we expected the columns to be, so it's treated them all as 'objects' (basically: strings) and not as numeric data types.

So we need to fix that now... as you saw before, there's a function called `'astype'` that allows us to convert between data types where it's fairly easy for pandas to figure out what we want to do:

In [ ]:
df3.Temperature.describe()

In [ ]:
for c in ['WindDirection','WeatherType','PressureTendency']:
    df3[c] = df3[c].astype('category')
for c in ['DewPoint','Humidity','Temperature']:
    df3[c] = df3[c].astype('float')
for c in ['WindGust','Visibility']:
    df3[c] = df3[c].astype('int')

In [ ]:
df3.Temperature.describe()

That's more like it!

### Working with Timeseries Data

So that's looking a lot more useful, but as a final step we need to make sure that the temporal data is actually treated as a time series... again, Google is your friend here: `"pandas convert datetime to time series"`. 

Given that we are creating a _new_ column called 'Time' from an _existing_ column called 'ts', what do you think needs to replace the '???'?

In [ ]:
df3['Time'] = pd.to_datetime(df3.???.values, infer_datetime_format=True)

And now compare:

In [ ]:
df3.ts.head()

In [ ]:
df3.Time.head() # A quick check

## Using a Time Series in an Index

We can tell that that type conversion succeeded because we've got a new `dtype`: `datetime64[ns]`. 

We can also now do some really neat things to 'resample' the data based on the fact that we have temporal data; however, to take advantage of this we have to let pandas know that the entire data set is organised by time. We do this by replacing the existing integer index with a datetime one:

In [ ]:
df3.index = pd.to_datetime(df3.ts.values, infer_datetime_format=True)
df3.head(3)

In the output above, you'll notice that the left-most column (the one without a name, because it's an _index_, not a column) is now a datetime object. Why is that useful? Well check _this_ out:

In [ ]:
df3.Temperature.resample('D').mean()

OK, this is another geeky moment but how cool is that? By telling pandas that our data is temporal, we're now in a position to ask pandas to answer questions like "What was the average daily ('D') temperature at Heathrow?" And we can do this in _one_ line of code.

If you had data at minutes-level resolution, then you could aggregate to Hourly or Daily. In principle, you can also do all sorts of datetime queries around things like "What was the weekly average in the 3rd week of 2016?" or "What was last Friday's weather?".

# Plotting!

This has been a long, slow build towards something more exciting: plotting! Well, plotting _again_. In a way, this has been a lot of effort just to make a graph, but let's recognise where we're at:

* We can request data for _any_ lcoation in Britain by changing the location id.
* We can get new data _any_ time we feel like it.
* We can (in a minute) create a plot of that data.
* We can update it continuously in the future!

That's pretty awesome, right?

In [ ]:
# This command tells Jupyter that we want 
# the plots to be shown inline (on this 
# web page). You'll always need to do this
# *once* on a notebook.
%matplotlib inline

Pandas can do a _lot_ of different plots, [see for yourself](http://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization-hexbin). Here's a sampling:

In [ ]:
df3.Humidity.plot()

In [ ]:
df3.Temperature.plot.bar()

In [ ]:
df3.Temperature.plot.box() # Handy!

In [ ]:
df3.plot.scatter(x='Temperature', y='Humidity') # Spot the problem data point

In [ ]:
df3.plot.scatter(x='Temperature', y='Humidity', s=(df3.DewPoint+1)*25);

In [ ]:
df3.plot.hexbin(x='Temperature', y='Humidity', gridsize=15)

In [ ]:
df3.WindDirection.plot() # Ooops. Can you tell what is going wrong? Google it.

## Wrap-up

In this workshop, you have accessed the weather data via an API. This technique can be used in a wide range of applications such as fetching the Point of Interesting (POI) data via Google Places API. 

You have also explored a new dataset using various function in the Pandas package.

## Credits!

### Acknowledgements:

This workshop is largely based on a [notebook](https://github.com/jreades/i2p/blob/master/practicals/Practical-X-API_Data.ipynb) developed by [Jon Reades](https://github.com/jreades). 